In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

dataset_path = r"C:\Users\Bobby\Documents\Animal_Classification\Animal Classification\dataset"


train_dataset = image_dataset_from_directory(dataset_path,
                                             validation_split=0.2, 
                                             subset="training",
                                             seed=123,
                                             image_size=(224, 224),
                                             batch_size=32)

val_dataset = image_dataset_from_directory(dataset_path,
                                           validation_split=0.2, 
                                           subset="validation",
                                           seed=123,
                                           image_size=(224, 224),
                                           batch_size=32)


class_names = train_dataset.class_names
print(class_names)


In [ ]:
from tensorflow.keras import layers


normalization_layer = layers.Rescaling(1./255)


data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2)
])


train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout


base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')


base_model.trainable = False


model = Sequential([
    data_augmentation,  
    base_model,  
    GlobalAveragePooling2D(),  
    Dense(128, activation='relu'),
    Dropout(0.3),  
    Dense(15, activation='softmax')  
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=10)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image


dataset_path = r"C:\Users\Bobby\Documents\Animal_Classification\Animal Classification\dataset"

class_folders = os.listdir(dataset_path)

for class_folder in class_folders:
    class_path = os.path.join(dataset_path, class_folder)

    if not os.path.isdir(class_path):
        continue
        
    image_files = [file for file in os.listdir(class_path) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for img_name in image_files:
        img_path = os.path.join(class_path, img_name)  
      
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

       
        prediction = model.predict(img_array)
        pred_class = class_names[np.argmax(prediction)]

        print(f"🔹 Processing Image: {img_path} | Predicted Class: {pred_class}")

       
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"Actual: {class_folder}\nPredicted: {pred_class}")
        plt.show()


In [ ]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)  
model.save("animal_model.keras")  


In [ ]:
model.save("animal_model.h5", save_format="h5")
